# Análisis Ecommerce - Limpieza
### *Javier Ramírez*
---

## Índice

1. Importaciones
2. Extracción de datos
3. Unión de datos
4. Diseño del proyecto
5. Limpieza de datos

---

## 1. Importaciones

In [1]:
import warnings
# Ignorar todas las advertencias
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

---

## 2. Extracción de Datos

In [2]:
meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Extraer el archivo de cada mes y guardarlo en una variable con su nombre
for mes in meses:
    exec(f'{mes.lower()[:3]} = pd.read_csv("Datos/Sales_{mes}_2019.csv")')

# Visualizar la estructura de cada dataset
for mes in meses:
    print(mes + ':' + str(eval(mes.lower()[:3]).shape))

# Crear una lista con los nombres de cada dataset
meses_r = []
for mes in meses:
    meses_r.append(mes.lower()[:3])

January:(9723, 6)
February:(12036, 6)
March:(15226, 6)
April:(18383, 6)
May:(16635, 6)
June:(13622, 6)
July:(14371, 6)
August:(12011, 6)
September:(11686, 6)
October:(20379, 6)
November:(17661, 6)
December:(25117, 6)


---

## 3. Unión de Datos

In [3]:
jan.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


In [4]:
feb.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,150502,iPhone,1,700,02/18/19 01:35,"866 Spruce St, Portland, ME 04101"
1,150503,AA Batteries (4-pack),1,3.84,02/13/19 07:24,"18 13th St, San Francisco, CA 94016"
2,150504,27in 4K Gaming Monitor,1,389.99,02/18/19 09:46,"52 6th St, New York City, NY 10001"
3,150505,Lightning Charging Cable,1,14.95,02/02/19 16:47,"129 Cherry St, Atlanta, GA 30301"
4,150506,AA Batteries (4-pack),2,3.84,02/28/19 20:32,"548 Lincoln St, Seattle, WA 98101"


In [5]:
# Concatenar todos los datos de cada mes
df = pd.concat([globals()[mes] for mes in meses_r], axis = 0).reset_index(drop = True)
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
...,...,...,...,...,...,...
186845,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016"
186846,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001"
186847,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101"
186848,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001"


Apilamos los registros usando pd.concat() con el axis = 0, para que lo junte por filas y no por columnas y usamos globals() para acceder a las variables globales, ya que los elementos de meses_r estan en formato string y queremos que sean las variables que contienen los dataframes.

In [6]:
df['Order ID'].duplicated().sum()

8411

In [7]:
dupl = df.loc[df['Order ID'].duplicated(keep = False)]
dupl.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
41,141275,USB-C Charging Cable,1,11.95,01/07/19 16:06,"610 Walnut St, Austin, TX 73301"
42,141275,Wired Headphones,1,11.99,01/07/19 16:06,"610 Walnut St, Austin, TX 73301"
57,141290,Apple Airpods Headphones,1,150,01/02/19 08:25,"4 1st St, Los Angeles, CA 90001"
58,141290,AA Batteries (4-pack),3,3.84,01/02/19 08:25,"4 1st St, Los Angeles, CA 90001"
133,141365,Vareebadd Phone,1,400,01/10/19 11:19,"20 Dogwood St, New York City, NY 10001"


Vemos que en algunas ocasiones el Order ID se repite, queriendo decir que este hace referencia o bien a un cliente o a una sesión de compra, vamos a comprobarlo.

In [8]:
s = dupl.groupby(['Order ID', 'Order Date'], as_index = False)[['Purchase Address']].max()
s.head()

,Order ID,Order Date,Purchase Address
0,141275,01/07/19 16:06,"610 Walnut St, Austin, TX 73301"
1,141290,01/02/19 08:25,"4 1st St, Los Angeles, CA 90001"
2,141365,01/10/19 11:19,"20 Dogwood St, New York City, NY 10001"
3,141384,01/03/19 00:14,"223 Jackson St, Boston, MA 02215"
4,141450,01/12/19 11:16,"521 Park St, San Francisco, CA 94016"


In [9]:
s['Order ID'].duplicated().sum()

0

Vemos que para cada Order ID solo hay una fecha, por lo que se trata de la sesión, ya que si fuesen clientes habrian varias fechas, a no ser que ningún cliente haya realizado mas de una compra, algo bastante raro.

---

## 4. Diseño del proyecto

### 4.1. Objetivo

El objetivo de este proyecto es proponer acciones para incrementar la facturación de la empresa, ya sea aumentando las ventas o disminuyendo los costes.

### 4.2. Palancas

1. Cliente
2. Producto
3. Sesion
4. Localizacion

### 4.3. KPIs

#### Clientes:
    1. Número de compras/año
    2. Número de productos comprados/año
    3. Dinero gastado/año

#### Producto:
    1. Productos comprados/año
    2. Productos comprados/mes
    3. Media de productos diferentes comprados por cada cliente

#### Sesión:
    1. Número de productos comprados por sesión
    2. Dinero gastado por sesión

#### Localización:
    1. Estados y ciudades con más compras
    2. Estados y ciudades con más dinero gastado
    3. Estados y ciudades con menos compras
    4. Estados y ciudades con menos dinero gastado

### 4.4. Preguntas semilla

#### Cliente:
    1. ¿Cuántos productos compra cada cliente en promedio?
    2. ¿Cuánto gasta cada cliente mensual y anualmente?
    3. ¿Existen "mejores clientes" que debamos identificar y tratar de manera diferenciada?
    4. ¿Los clientes repiten sus compras en los meses siguientes?
    5. ¿Cuál es el valor de vida promedio (LTV) de un cliente?

#### Producto:
    1. ¿Cuáles son los productos más vendidos?
    2. ¿Existen productos que tienen una venta prácticamente nula?
    3. ¿Hay productos que son adquiridos regularmente por el mismo cliente?
    4. ¿Existen productos que son comprados en grandes cantidades en una sola compra?
    5. ¿Existe una relación entre el precio y el volumen de ventas?
    6. ¿Existen meses en los que se adquieren más o menos productos?
    7. ¿Es posible generar recomendaciones de productos personalizadas para cada cliente?

#### Sesión:
    1. ¿Cuántos productos se compran en promedio por sesión?
    2. ¿Cuánto dinero se gasta en promedio por sesión?
    3. ¿A qué horas suelen realizarse las compras?
    4. ¿Existen días concretos en los que se ha observado un aumento significativo en el número de ventas?

#### Localización:
    1. KPIs
    2. ¿Existe alguna relación entre los estados o ciudades y alguna otra variable?

---

## 5. Limpieza de Datos

In [10]:
print(df.shape)
df.head()

(186850, 6)


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


### 5.1. Valores nulos

In [12]:
df.isna().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [13]:
df.loc[df.isnull().any(axis=1)].head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
664,NaN,NaN,NaN,NaN,NaN,NaN
678,NaN,NaN,NaN,NaN,NaN,NaN
797,NaN,NaN,NaN,NaN,NaN,NaN
876,NaN,NaN,NaN,NaN,NaN,NaN
1299,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Eliminar resgistros con nulos
df = df.dropna()
df.shape

(186305, 6)

In [15]:
print('Número de nulos: ', df.isna().sum().sum())

Número de nulos:  0


### 5.2. Tipos de datos

Variables que hay que cambiar el tipo:
- Order ID -> Int
- Quantity Ordered -> Int
- Price Each -> Float
- Order Date -> Datetime

Antes de cambiar los tipos voy a cambiar los nombres de las variables y ponerlas en minusculas y sin espacios para trabajar más facilmente.

In [16]:
df.columns = ['order_id', 'product', 'quantity_ordered', 'price_each', 'order_date',
       'purchase_address']

In [17]:
df['order_id'] = pd.to_numeric(df['order_id'], errors='coerce').astype('Int64')
df['quantity_ordered'] = pd.to_numeric(df['quantity_ordered'], errors='coerce').astype('Int64')
df['price_each'] = pd.to_numeric(df['price_each'], errors='coerce').astype('Float64')

In [18]:
# Al ejecutar este código sale el siguiente error: 'Unknown string format: Order Date present at position 1069'
# pd.to_datetime(df['order_date'])

df.iloc[[1069], :]

,order_id,product,quantity_ordered,price_each,order_date,purchase_address
1073,<NA>,Product,<NA>,<NA>,Order Date,Purchase Address


In [19]:
df.query('order_date == "Order Date"').shape

(355, 6)

In [20]:
# Eliminar registros incorrectos
eliminar = df.query('order_date == "Order Date"').index.to_list()
df = df.drop(index = eliminar)

# Cambiar el tipo de la variable
df['order_date'] = pd.to_datetime(df['order_date'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185950 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   order_id          185950 non-null  Int64         
 1   product           185950 non-null  object        
 2   quantity_ordered  185950 non-null  Int64         
 3   price_each        185950 non-null  Float64       
 4   order_date        185950 non-null  datetime64[ns]
 5   purchase_address  185950 non-null  object        
dtypes: Float64(1), Int64(2), datetime64[ns](1), object(2)
memory usage: 10.5+ MB


### 5.3. Registros duplicados

In [22]:
print('Número de registros duplicados: ', df.duplicated().sum())

Número de registros duplicados:  264


In [23]:
df[df.duplicated(keep=False)]

,order_id,product,quantity_ordered,price_each,order_date,purchase_address
874,142071,AA Batteries (4-pack),1,3.84,2019-01-17 23:02:00,"131 2nd St, Boston, MA 02215"
875,142071,AA Batteries (4-pack),1,3.84,2019-01-17 23:02:00,"131 2nd St, Boston, MA 02215"
4125,145143,Lightning Charging Cable,1,14.95,2019-01-06 03:01:00,"182 Jefferson St, San Francisco, CA 94016"
4126,145143,Lightning Charging Cable,1,14.95,2019-01-06 03:01:00,"182 Jefferson St, San Francisco, CA 94016"
5810,146765,Google Phone,1,600.0,2019-01-21 11:23:00,"918 Highland St, New York City, NY 10001"
...,...,...,...,...,...,...
182973,315955,ThinkPad Laptop,1,999.99,2019-12-26 17:28:00,"588 Chestnut St, Seattle, WA 98101"
183199,316173,AAA Batteries (4-pack),1,2.99,2019-12-22 22:44:00,"907 Sunset St, Portland, OR 97035"
183200,316173,AAA Batteries (4-pack),1,2.99,2019-12-22 22:44:00,"907 Sunset St, Portland, OR 97035"
185084,317971,AA Batteries (4-pack),1,3.84,2019-12-17 18:39:00,"250 Chestnut St, San Francisco, CA 94016"


En este caso se pueden haber generado los duplicados sin querer y podria ser que el cliente haya comprado dos unidades del mismo producto en la misma sesión pero por algún motivo se haya dividido en dos registros diferentes. Pero como hay también compras de ordenadores de 999.99$ supongo que se habran generado los duplicados por error y no sucede lo que he comentado.

In [24]:
# Eliminar registros duplicados
df = df.drop_duplicates()

# Comprobar duplicados
print('Número de registros duplicados: ', df.duplicated().sum())

Número de registros duplicados:  0


In [25]:
# Resetear el índice
df = df.reset_index(drop = True)

### 5.4. Revisar los datos

In [26]:
df.describe(include = 'all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
order_id,185686.0,<NA>,<NA>,<NA>,230411.376227,141234.0,185833.25,230354.5,275028.75,319670.0,51511.717183
product,185686,19,USB-C Charging Cable,21859,NaN,NaN,NaN,NaN,NaN,NaN,NaN
quantity_ordered,185686.0,<NA>,<NA>,<NA>,1.124544,1.0,1.0,1.0,1.0,9.0,0.443069
price_each,185686.0,<NA>,<NA>,<NA>,184.519255,2.99,11.95,14.95,150.0,1700.0,332.843838
order_date,185686,NaN,NaN,NaN,2019-07-18 21:32:06.298051328,2019-01-01 03:07:00,2019-04-16 20:55:15,2019-07-17 20:11:00,2019-10-26 08:00:00,2020-01-01 05:13:00,NaN
purchase_address,185686,140787,"193 Forest St, San Francisco, CA 94016",9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Parece que no hay ninguna cosa rara en los datos.

Conclusiones:
- Sólo hay 19 productos en el catálogo, o por lo menos son los que se han comprado.
- El producto más vendido (21480 ventas) es el USB-C Charging Cable.
- Han habido un máximo de 9 compras de un mismo cliente en todo el año.

### 5.5. Guardar el dataset

In [27]:
pd.to_pickle(df, 'df.pickle')